### 사용 DB생성
- MYSQL client 이용

- DROP DATABASE IF EXISTS student_mgmt;
- CREATE DATABASE student_mgmt DEFAULT CHARSET=utf8 COLLATE=utf8_bin;


* DEFAULT CHARSET=utf8 COLLATE=utf8_bin
  - 문자 인코딩(문자셋)과 관련된 설정 
    - 한글 데이터의 정확한 처리를 위해, 데이터베이스/테이블 생성시에 추가
    - 문자셋(Character set)은 각 문자가 컴퓨터에 저장될 때 어떠한 '코드'로 저장될지에 대한 규칙의 집합을 의미
  - 콜레이션(Collation)은 특정 문자 셋에 의해 데이터베이스에 저장된 값들을 비교 검색하거나 정렬 등의 작업을 위해 문자들을 서로 '비교' 할때 사용하는 규칙들의 집합을 의미


In [ ]:
USE student_mgmt;
DROP TABLE IF EXISTS students;


CREATE TABLE students (
  id TINYINT NOT NULL AUTO_INCREMENT,
  name VARCHAR(10) NOT NULL,
  gender ENUM('man','woman') NOT NULL,
  birth DATE NOT NULL,
  english TINYINT NOT NULL,
  math TINYINT NOT NULL,
  korean TINYINT NOT NULL,
  PRIMARY KEY (id)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;


- ENUM 타입: 특정 값만 저장할 수 있도록 저장 가능 값을 지정해주는 데이터 타입
- ENGINE=InnoDB : 스토리지 엔진이 여러 가지 있음 이 중에서 InnoDB로 스토리지 엔진을 정해주는 명령 이 명령을 안해줘도 최근 Mysql 버전은 디폴트로 엔진을 InnoDB를 쓴다고 함
- 테이블을 만들때도 DEFAULT CHARSET=utf8 인코딩


In [28]:
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('dave', 'man', '1983-07-16', 90, 80, 71);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('minsun', 'woman', '1982-10-16', 30, 88, 60);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('david', 'man', '1982-12-10', 78, 77, 30);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('jade', 'man', '1979-11-1', 45, 66, 20);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('jane', 'man', '1990-11-12', 65, 32, 90);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('wage', 'woman', '1982-1-13', 76, 30, 80);
INSERT INTO students (name, gender, birth, english, math, korean) VALUES ('tina', 'woman', '1982-12-3', 87, 62, 71);


#### pandas 라이브러리와 pymysql

1. read_sql()
    - sql 연결 객체를 활용하여 쿼리 구문으로 반환된 결과를데이터 프레임으로 바로 생성해주는 함수

In [2]:
import pymysql
import pandas as pd

In [4]:
host_name = 'localhost'
host_port = 3307
username = 'root'
password = '0000'
database_name = 'student_mgmt'


In [5]:
# db연결
db = pymysql.connect(
    host=host_name,     # MySQL Server Address
    port=host_port,          # MySQL Server Port
    user=username,      # MySQL username
    passwd=password,    # password for MySQL username
    db=database_name,   # Database name
    charset='utf8'
)


In [22]:
sql = 'select * from students'

In [23]:
df = pd.read_sql(sql, db)

In [24]:
df

,id,name,gender,birth,english,math,korean
0,1,dave,man,1983-07-16,90,80,71
1,2,minsun,woman,1982-10-16,30,88,60
2,3,david,man,1982-12-10,78,77,30
3,4,jade,man,1979-11-01,45,66,20
4,5,jane,man,1990-11-12,65,32,90
5,6,wage,woman,1982-01-13,76,30,80
6,7,tina,woman,1982-12-03,87,62,71


In [26]:
type(df['math'][0])

numpy.int64

In [27]:
df.to_csv('./DATA_CSV/students.csv',sep=',',index=False,encoding='utf-8')

#### 외래키(FOREIGN KEY)를 만드는 이유
* 두 테이블 사이에 관계를 선언해서, 데이터의 무결성을 보장


In [30]:
import pymysql
import pandas as pd


In [31]:
host_name = 'localhost'
host_port = 3307
username = 'root'
password = '0000'
database_name = 'sqlDB'


In [32]:
db = pymysql.connect(
    host=host_name,     # MySQL Server Address
    port=host_port,          # MySQL Server Port
    user=username,      # MySQL username
    passwd=password,    # password for MySQL username
    db=database_name,   # Database name
    charset='utf8'
)


In [34]:
sql = "select * from userTbl"
df = pd.read_sql(sql,db)
df

,userID,name,birthYear,addr,mobile1,mobile2,height,mDate
0,BBK,바비킴,1973,서울,010,00000000,176,2013-05-05
1,EJW,은지원,1972,경북,011,88888888,174,2014-03-03
2,JKW,조관우,1965,경기,016,99999999,172,2010-10-10
3,JYP,조용필,1950,경기,011,44444444,166,2009-04-04
4,KBS,김범수,1979,경남,011,22222222,173,2012-04-04
5,KKH,김경호,1971,전남,019,33333333,177,2007-07-07
6,LJB,임재범,1963,서울,016,66666666,182,2009-09-09
7,LSG,이승기,1987,서울,011,11111111,182,2008-08-08
8,SSK,성시경,1979,서울,None,None,186,2013-12-12
9,YJS,윤종신,1960,경남,None,None,170,2005-05-05


#### buyTbl에 데이터를 추가

- 외래키로 지정되어 있는 userID에 입력되는 새로운 값 STJ가 userTbl에 없는 값이어서 무결성 오류 발생

In [40]:
cursor = db.cursor()
sql_query = "INSERT INTO buyTbl (userID, prodName, groupName, price, amount) VALUES('STJ', '운동화', '의류', 30, 2);"
cursor.execute(sql_query)
db.commit()

InterfaceError: (0, '')

#### 위의 셀 에러가 나면 정상임
* CONSTRAINT `buyTbl_ibfk_1` FOREIGN KEY (`userID`) REFERENCES `userTbl` (`userID`)
* userTbl 에 userID가 STJ인 데이터가 없기 때문에, 
  - FOREIGN KEY (userID) REFERENCES userTbl(userID)
  - buyTbl 테이블의 userID 커럼은 userTbl 테이블의 userID를 참조할 때, userTbl 테이블에 userID가 STJ인 데이터가 없으면, 입력이 안됨
  - 데이터 무결성 (두 테이블간 관계에 있어서, 데이터의 정확성을 보장하는 제약 조건을 넣는 것임)
  - 현업에서는 꼭 필요한 경우만 사용하는 경우가 많음 (비즈니스 로직이 다양하기 때문에, 제약을 걸어놓을 경우, 예외적인 비즈니스 로직 처리가 어렵기 때문)


In [41]:
cursor = db.cursor()
SQL_QUERY = "INSERT INTO buyTbl (userID, prodName, groupName, price, amount) VALUES('BBK', '운동화', '의류', 30, 2);"
cursor.execute(SQL_QUERY)
db.commit()


InterfaceError: (0, '')

In [38]:
### 에러발생시 db.close()로 닫아주고, 다시 connect

In [42]:
db.close()

Error: Already closed

In [47]:
def conn(d_name) :
    import pymysql
    host_name = 'localhost'
    host_port = 3307
    username = 'root'
    password = '0000'
    database_name = d_name
    db = pymysql.connect(
        host=host_name,     # MySQL Server Address
        port=host_port,          # MySQL Server Port
        user=username,      # MySQL username
        passwd=password,    # password for MySQL username
        db=database_name,   # Database name
        charset='utf8'
    )
    return db


In [48]:
db = conn('sqlDB')

#### 이번에는 userTbl 에 userID가 STJ 인 데이터를 넣어준 후에, 다시 buyTbl userID에 STJ 관련 데이터를 넣어줍니다.


In [54]:
cursor = db.cursor()
sql_query = "INSERT INTO userTbl VALUES('STJ', '서태지', 1975, '경기','011','00000000',171,'2014-4-4')"
cursor.execute(sql_query)
db.commit()

In [55]:
SQL_QUERY = "INSERT INTO buyTbl (userID, prodName, groupName, price, amount) VALUES('STJ', '운동화', '의류', 30, 2);"
cursor.execute(SQL_QUERY)
db.commit()

#### 이번에는 userTbl에 userID가 STJ 관련 데이터를 삭제해봅니다.


In [56]:
sql_query = "delete from userTbl where userID='STJ'"
cursor.execute(sql_query)
db.commit()

IntegrityError: (1451, 'Cannot delete or update a parent row: a foreign key constraint fails (`sqldb`.`buytbl`, CONSTRAINT `buytbl_ibfk_1` FOREIGN KEY (`userID`) REFERENCES `usertbl` (`userID`))')

#### 에러나면 정상입니다.
* buyTbl 에 해당 userID를 외래키로 참조하는 데이터가 있기 때문입니다.
